In [11]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
import glob
from src.walking_recording import WalkingRecording
from typing import Dict
import matplotlib.pyplot as plt

In [12]:
def setup() ->  Dict:
    """
    Fonction de setup de l'application
    Permet de récupérer les données des membres du groupe

    Returns:
        Dict: Dictionnaire contenant les données des membres
    """
    file_path = 'data/processed'
    members_list = glob.glob(f"{file_path}/*")
    members_dict = {}

    for member in members_list:
        member_name = os.path.basename(member)
        members_dict[member_name] = {}
        file_list = glob.glob(f"{member}/*")
        for csv_file in file_list:
            basename = os.path.basename(csv_file)
            members_dict[member_name][basename] = WalkingRecording.from_csv(csv_file)

    return members_dict


In [13]:
members = setup()
list(members.keys())

['Felix', 'Antoine', 'Leo', 'Matthieu', 'Corentin', 'Serge']

In [14]:
def members_metrics_df(members):
    metrics = []
    for name, member in members.items():
        for file_name, recording in member.items():
            metrics.append({
                "name" : name,
                "recording" : file_name,
                "steps" : len(recording.get_steps()),
                "frequency_fft" : recording.get_frequency_from_fft(),
                "frequency_steps" : recording.get_frequency_from_counting_steps(),
                "acceleration_std" : recording.get_std("acceleration (g)")
            })
    return pd.DataFrame(metrics)
        

In [15]:
leo = members["Leo"]["accelerometer1_data.csv"]
autocorr = leo.get_autocorrelation()
sp = leo.get_sampling_period()
f0 = 1/(sp*np.argmax(np.abs(autocorr)))
f0

np.float64(0.09910802775024777)